In [1]:
# step 5 - run bed tools

In [2]:
# sort tagAlign file
# install bedtools:
# conda install -c bioconda bedtools
# conda install -c bioconda htslib

!pwd

/Users/eilaarich-landkof-stanford/Documents/Code/git_kundaje_genomics/DNA/ATAC/single-cell/fragments-file-manipulation/synapse_in_out


In [3]:
clusters_fld = "clusters"

In [4]:
import os
chrom_sizes_fld = "chrom_sizes_file"
os.makedirs(chrom_sizes_fld,exist_ok=True)
# TODO: change it to your relevant chromosome size
gs_chrom_sizes_file_path = "gs://broad-buenrostro-pipeline-genome-annotations/IGVF_human_v43/GRCh38_EBV.chrom.sizes.tsv"
local_chrom_sizes_file_path = os.path.join(os.getcwd(),chrom_sizes_fld,os.path.basename(gs_chrom_sizes_file_path))
print("local_chrom_sizes_file_path is {}".format(local_chrom_sizes_file_path))
# download the relevant chormosome for your needs:
if (os.path.exists(local_chrom_sizes_file_path) == False):
    !gsutil cp $gs_chrom_sizes_file_path $local_chrom_sizes_file_path
else:
    print("Nothing to download. local_chrom_sizes_file_path {} exists".format(local_chrom_sizes_file_path))


local_chrom_sizes_file_path is /Users/eilaarich-landkof-stanford/Documents/Code/git_kundaje_genomics/DNA/ATAC/single-cell/fragments-file-manipulation/synapse_in_out/chrom_sizes_file/GRCh38_EBV.chrom.sizes.tsv
Nothing to download. local_chrom_sizes_file_path /Users/eilaarich-landkof-stanford/Documents/Code/git_kundaje_genomics/DNA/ATAC/single-cell/fragments-file-manipulation/synapse_in_out/chrom_sizes_file/GRCh38_EBV.chrom.sizes.tsv exists


In [5]:
# this one is only for validation. it is not being used to generate the tagAlign file
import pandas as pd
local_sorted_chrom_sizes_file_path = local_chrom_sizes_file_path.split(".tsv")[0]+".sorted.tsv"
sorted_df = None
if os.path.exists(local_sorted_chrom_sizes_file_path) == False:
    # Step 1: Load the chromosome size file into a DataFrame
    df = pd.read_csv(local_chrom_sizes_file_path,sep="\t", header=None)
    print("df before sort is {}".format(df.head()))
    # Step 2: Sort the DataFrame by the second column
    sorted_df = df.sort_values(by=df.columns[1])
    # print("df after sort is {}".format(sorted_df.head())
    
    # Step 3: Save the sorted DataFrame to a new TSV file
    output_file_path = local_chrom_sizes_file_path.split(".tsv")[0]+".sorted.tsv"
    sorted_df.to_csv(local_sorted_chrom_sizes_file_path, sep='\t', index=False, header = False)
    
    # Step 4: Confirm successful save
    print(f"Sorted DataFrame saved to {output_file_path}")
else:
    sorted_df = pd.read_csv(local_sorted_chrom_sizes_file_path,sep="\t", header=None)
    print("sorted file: {} is available".format(local_sorted_chrom_sizes_file_path))

sorted file: /Users/eilaarich-landkof-stanford/Documents/Code/git_kundaje_genomics/DNA/ATAC/single-cell/fragments-file-manipulation/synapse_in_out/chrom_sizes_file/GRCh38_EBV.chrom.sizes.sorted.tsv is available


In [6]:
import os
list_of_folders_in_clusters = os.listdir(os.path.join(os.getcwd(),clusters_fld))
list_of_folders_in_clusters = [x for x in list_of_folders_in_clusters if ".ipynb" not in x and ".DS_Store" not in x]
list_of_folders_in_clusters[0:5]

['ENCSR052DKH', 'ENCSR397LYX', 'ENCSR367GKP', 'ENCSR205TUH', 'ENCSR868ZEI']

In [7]:
import subprocess

def run_bedtools_command(command):
    try:
        # Execute the Bedtools command and capture the output and errors
        result = subprocess.run(command, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        if result.stderr:
            print("Error occurred:")
            print(result.stderr)
        else:
            print("Output:")
            print(result.stdout)
    except subprocess.CalledProcessError as e:
        print("Error occurred:")
        print(e.stderr)


In [8]:
# list_of_folders_in_clusters=['ENCSR023FME'] # debug
# list_of_folders_in_clusters

In [10]:
# "tagAlign_" in f and "CellClusterAssignment" not in f

In [9]:
# os.listdir(os.path.join(clusters_fld,fld))
list_of_folders_in_clusters[0:1]

['ENCSR052DKH']

In [11]:
files_list

['ENCSR052DKH_Cluster0.tsv.tagAlign',
 'ENCSR052DKH_Cluster1.tsv.tagAlign',
 'ENCSR052DKH_ClusterMetadata.tsv',
 'ENCSR052DKH_Cluster3.tsv.tagAlign',
 'ENCSR052DKH_Cluster2.tsv.tagAlign']

In [12]:
#debug
# for fld in list_of_folders_in_clusters[0:1]:
for fld in list_of_folders_in_clusters:
    files_list = os.listdir(os.path.join(clusters_fld,fld))
    files_list = [x for x in files_list if "DS_Store" not in x and ".ipynb" not in x and "sort_tagAlign_" not in x and "cluster_id_annotation_map" not in x]
    for f in files_list:
        # print("f is {}".format(f))
        f_full_path = os.path.join(os.getcwd(),clusters_fld,fld,f)
        print("\n\nf_full_path is {}".format(f_full_path))
        if ("tagAlign_" in f or ".tagAlign" in f) and "stats" not in f:
            sort_fld = "sort_{}".format(f.split(".tsv")[0])
            
            print("*****run bedtools for {}".format(f_full_path))
            output_tagAlign_gz_folder = os.path.join(os.getcwd(),clusters_fld,fld,sort_fld)
            os.makedirs(output_tagAlign_gz_folder, exist_ok=True)
            output_sorted_tagAlign = os.path.join(output_tagAlign_gz_folder,f.split(".tsv")[0])+".sorted.tagAlign"
            output_sorted_tagAlign_gz = output_sorted_tagAlign+".gz"
            if os.path.exists(output_sorted_tagAlign_gz):
                #####output_sorted_tagAlign_gz /Users/eilaarich-landkof-stanford/Documents/Code/git_kundaje_genomics/DNA/ATAC/single-cell/fragments-file-manipulation/synapse_in_out/clusters/ENCSR023FME/tagAlign_ENCSR023FME_Endothelial_cell.tsv exists
                print("#####output_sorted_tagAlign_gz {} exists. continue".format(f_full_path))
                continue
            # print("output_tagAlign_gz is {}".format(output_tagAlign_gz))
            # print("bedtools sort -i {} -g {} > {}".format(f_full_path,
            #                                               local_chrom_sizes_file_path,  #local_sorted_chrom_sizes_file_path,
            #                                               output_tagAlign))

            # bedtools sort -i input.tagalign -g chromosome_sizes.txt > output_sorted.tagalign | bgzip -c > output_sorted.tagalign.gz && tabix -p bed output_sorted.tagalign.gz
            bedtools_command = "bedtools sort -i {} -g {} | bgzip -c > {} && tabix -p bed {}".format(f_full_path,
                                                                                                     local_chrom_sizes_file_path, 
                                                                                                     output_sorted_tagAlign_gz,
                                                                                                     output_sorted_tagAlign_gz)

            print("bedtools_command is {}".format(bedtools_command))
            run_bedtools_command(bedtools_command)
        else:
            print("\n\n!!!Will not run bedtools for file: {}".format(f_full_path))

# from Jesse
# bedtools sort -faidx {the IGVF chrom sizes file} {input.tagAlign} | bgzip > {output.tagAlign.gz}; tabix -p bed {output.tagAlign.gz})





f_full_path is /Users/eilaarich-landkof-stanford/Documents/Code/git_kundaje_genomics/DNA/ATAC/single-cell/fragments-file-manipulation/synapse_in_out/clusters/ENCSR052DKH/ENCSR052DKH_Cluster0.tsv.tagAlign
*****run bedtools for /Users/eilaarich-landkof-stanford/Documents/Code/git_kundaje_genomics/DNA/ATAC/single-cell/fragments-file-manipulation/synapse_in_out/clusters/ENCSR052DKH/ENCSR052DKH_Cluster0.tsv.tagAlign
bedtools_command is bedtools sort -i /Users/eilaarich-landkof-stanford/Documents/Code/git_kundaje_genomics/DNA/ATAC/single-cell/fragments-file-manipulation/synapse_in_out/clusters/ENCSR052DKH/ENCSR052DKH_Cluster0.tsv.tagAlign -g /Users/eilaarich-landkof-stanford/Documents/Code/git_kundaje_genomics/DNA/ATAC/single-cell/fragments-file-manipulation/synapse_in_out/chrom_sizes_file/GRCh38_EBV.chrom.sizes.tsv | bgzip -c > /Users/eilaarich-landkof-stanford/Documents/Code/git_kundaje_genomics/DNA/ATAC/single-cell/fragments-file-manipulation/synapse_in_out/clusters/ENCSR052DKH/sort_ENC

In [13]:
# df_output_tagAlign.head()

In [14]:
# len(first_column_values)

# For testing purpose only

In [15]:
# read the sorted tagAlign file and make sure that the sorting is correct
# df_output_tagAlign = pd.read_csv(output_tagAlign,sep="\t", header=None) # works for bedtools sort -i {} -g {} > {}

In [16]:
# first_column_values = df_output_tagAlign.iloc[:, 0]
# first_column_values[0:4]

In [17]:
# # test the sorting
# for index, val in enumerate(first_column_values[1000000:1500000]): #
#     # print(val)
#     # print(index)
#     next_val = first_column_values[index+1]
#     val_chrom_size = sorted_df[sorted_df.iloc[:, 0] == val]
#     # print(val_chrom_size)
#     next_val_chrom_size = sorted_df[sorted_df.iloc[:, 0] == next_val]
#     # print(next_val_chrom_size)
#     if val_chrom_size.iloc[0][1] <= next_val_chrom_size.iloc[0][1]:
#         continue
#     else:
#         print("bad for index {}".format(index))

In [18]:
# !pwd

In [19]:
# check the output gz file
# !zless /Users/eilaarich-landkof-stanford/Documents/Code/git_kundaje_genomics/DNA/ATAC/single-cell/fragments-file-manipulation/synapse_in_out/clusters/ENCSR023FME/sort_tagAlign_ENCSR023FME_Basal_epithelial_cell/tagAlign_ENCSR023FME_Basal_epithelial_cell.sorted.tagAlign.gz


In [20]:
# tagAlign_ENCSR023FME_Basal_epithelial_cell.tsv


